In [29]:
import duckdb
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import os
from pathlib import Path
import geopandas as gpd
import xml.etree.ElementTree as ET
import re


In [30]:
tree = ET.parse('RSS.xml')
root = tree.getroot()

print(tree)
items = []
for item in root.findall('./channel/item'):
    title = item.find('title').text.strip() if item.find('title') is not None else ""
    link = item.find('link').text.strip() if item.find('link') is not None else ""
    pub_date_raw = item.find('pubDate').text.strip() if item.find('pubDate') is not None else ""
    
    # Convertir fecha de publicación
    try:
        pub_date = datetime.strptime(pub_date_raw, "%a, %d %b %Y %H:%M:%S %Z")
    except ValueError:
        pub_date = None

    filename = link.split('/')[-1]
    lower_link = link.lower()
    lower_filename = filename.lower()

    # --- Lógica de Extracción de Metadatos ---

    # A) Categoría Principal (estudios_completos, estudios_basicos, etc.)
    main_category = "Otros"
    if "estudios_completos" in lower_link:
        main_category = "Estudios Completos"
    elif "estudios_basicos" in lower_link:
        main_category = "Estudios Basicos"
    elif "estudios_rutas" in lower_link:
        main_category = "Estudios de Rutas"
    elif "zonificacion" in lower_link:
        main_category = "Zonificacion"

    # B) Tipo de Estudio (viajes, etapas, etc.)
    study_type = "Desconocido"
    if "viajes" in lower_filename:
        study_type = "Viajes"
    elif "etapas" in lower_filename:
        if "_c" in lower_filename or "carretera" in lower_link:
            study_type = "Etapas (Carretera)"
        else:
            study_type = "Etapas"
    elif "pernoctaciones" in lower_filename:
        study_type = "Pernoctaciones"
    elif "personas" in lower_filename:
        study_type = "Personas"
    elif "frecuencia" in lower_link or "frecuencia" in lower_filename:
        study_type = "Frecuencia"
    elif "calidad" in lower_link or "descartados" in lower_filename:
        study_type = "Calidad"
    elif "od_rutas" in lower_filename:
        study_type = "Matriz OD Rutas"
    elif "relaciones_tramos" in lower_filename:
        study_type = "Relaciones Tramos-Rutas"
    elif "tramos_info" in lower_filename:
        study_type = "Info Tramos OD"
    elif "zonificacion" in lower_link:
        study_type = "Geometria/Zonificacion"
    elif "agregados" in lower_filename:
        study_type = "Datos Agregados"

    # C) Zona (municipios, distritos, GAU)
    zone_type = "N/A" # Por defecto
    if "municipios" in lower_link or "municipios" in lower_filename:
        zone_type = "Municipios"
    elif "distritos" in lower_link or "distritos" in lower_filename:
        zone_type = "Distritos"
    elif "gau" in lower_link or "gau" in lower_filename:
        zone_type = "GAU"
    elif "rutas" in lower_link:
        zone_type = "Rutas"
    
    # D) Extracción de Fecha (YYYY, MM, DD) desde el nombre del archivo
    # Buscamos patrones como 20240101 (diario) o 202401 (mensual)
    year, month, day = None, None, None
    
    # Regex para YYYYMMDD
    date_match_daily = re.search(r'(\d{4})(\d{2})(\d{2})', filename)
    # Regex para YYYYMM (archivos mensuales o tar)
    date_match_monthly = re.search(r'(\d{4})(\d{2})', filename)
    
    if date_match_daily:
        year = int(date_match_daily.group(1))
        month = int(date_match_daily.group(2))
        day = int(date_match_daily.group(3))
    elif date_match_monthly:
        year = int(date_match_monthly.group(1))
        month = int(date_match_monthly.group(2))
        # day se queda como None para mensuales

    items.append({
        "main_category": main_category, # Nueva columna solicitada
        "study_type": study_type,
        "zone_type": zone_type,
        "year": year,
        "month": month,
        "day": day,
        "publication_date": pub_date,
        "filename": filename,
        "source_url": link
    })
df_catalog = pd.DataFrame(items)

In [17]:
con = duckdb.connect('movilidad_datalake.duckdb')
con.sql("CREATE SCHEMA IF NOT EXISTS bronze")
con.sql("INSTALL spatial; LOAD spatial;")
con.sql("INSTALL httpfs; LOAD httpfs;")

IOException: IO Error: Cannot open file "c:\users\jorge\3-tier-data-lakehouse\movilidad_datalake.duckdb": El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso.

File is already open in 
C:\Users\Jorge\AppData\Local\Programs\Python\Python312\python.exe (PID 24620)

In [6]:
con.sql("""
    CREATE OR REPLACE VIEW bronze.catalog AS 
    SELECT * FROM parquet_scan('datalake/bronze/catalog/*.parquet')
""")

In [8]:
urls_febrero = con.sql("""
    SELECT source_url FROM bronze.catalog 
    WHERE year=2023 AND month=2 AND zone_type='Municipios' 
      AND main_category='Estudios Básicos' AND study_type='Viajes'
""").df()['source_url'].tolist()

In [17]:
def cargar_viajes_por_criterio(con, year, month, zone_type):
    """
    Busca en el catálogo y vuelca los datos directamente al DuckLake (Parquet)
    usando la estrategia incremental (APPEND).
    """
    print(f"--- Iniciando carga al Lake para: {zone_type} | {year}-{month:02d} ---")
    
    # 1. Consultar el Catálogo para obtener las URLs (Sin cambios aquí)
    files_df = con.sql(f"""
        SELECT source_url 
        FROM bronze.catalog 
        WHERE year = {year}
          AND month = {month}
          AND zone_type = '{zone_type}'
          AND main_category = 'Estudios Basicos'
          AND study_type = 'Viajes'
          AND filename LIKE '%.csv.gz'
    """).df()
    
    urls = files_df['source_url'].tolist()
    
    if not urls:
        print(f"⚠️ No se encontraron archivos para {zone_type} en {year}-{month}.")
        return

    # 2. Definir columnas variables según el tipo de zona (Mantenemos tu lógica)
    if zone_type == 'Municipios':
        col_origen = "origen"
        col_destino = "destino"
    else:
        col_origen = "origen"
        col_destino = "destino"

    # 3. EJECUTAR EL COPY (El cambio principal)
    try:
        # Usamos COPY ... TO ... (FORMAT PARQUET)
        # Usamos la opción APPEND para añadir ficheros sin borrar lo existente
        
        query = f"""
            COPY (
                SELECT 
                    -- 1. Transformación de Fecha (Corrección del VARCHAR '20230131')
                    strptime(fecha, '%Y%m%d')::DATE as date,
                    periodo as period,
                    CAST({col_origen} AS VARCHAR) as id_origin,
                    CAST({col_destino} AS VARCHAR) as id_destination ,
                    distancia as distance,
                    actividad_origen as activity_origin,
                    actividad_destino as activity_destination,
                    estudio_origen_posible as study_origin_pos,
                    estudio_destino_posible as study_destination_pos,
                    residencia as residence,
                    renta as rent,
                    edad as age,
                    sexo as sex,
                    viajes as n_trips,
                    viajes_km as trips_total_length_km,
                    -- 2. Metadatos (Tus columnas personalizadas)
                    {year} as part_year,
                    {month} as part_month,
                    '{zone_type}' as zone_type,  -- Valor fijo pasado por parámetro
                    'MITMA' as source,
                    filename as source_url,      -- Columna mágica generada por read_csv(..., filename=True)
                    current_timestamp as ingestion_date
                    
                FROM read_csv(
                    {urls}, 
                    delim='|', 
                    header=True, 
                    filename=True,
                    union_by_name=True,
                    null_padding=True,
                    ignore_errors=True,
                    all_varchar=True -- Leemos todo como texto para evitar fallos de tipo antes de castear
                )
            ) 
            TO 'datalake/bronze/viajes' 
            (
                FORMAT PARQUET, 
                PARTITION_BY (zone_type, source, part_year, part_month), 
                COMPRESSION 'ZSTD', 
                APPEND, -- ¡CRUCIAL! Esto permite añadir meses sin borrar los anteriores
                FILENAME_PATTERN 'data_{{uuid}}'
            )
        """
        
        con.sql(query)
        print(f" Éxito: Datos guardados en datalake/bronze/viajes (Parquet)")
        
    except Exception as e:
        print(f"Error durante la carga al Lake: {e}")



In [ ]:
# --- EJEMPLO DE USO ---
con = duckdb.connect('movilidad_datalake.duckdb')
con.sql("INSTALL httpfs; LOAD httpfs;")

# Cargar Enero 2023
cargar_viajes_por_criterio(con, year=2023, month=1, zone_type='Municipios')

# Si luego quieres cargar otro mes, simplemente llamas de nuevo:
# cargar_viajes_por_criterio(con, year=2023, month=2, zone_type='Municipios')



--- Iniciando carga al Lake para: Municipios | 2023-01 ---
 Éxito: Datos guardados en datalake/bronze/viajes (Parquet)


In [21]:
con = duckdb.connect('movilidad_datalake.duckdb')
con.sql("""
    CREATE OR REPLACE VIEW bronze.trips AS 
    SELECT * FROM parquet_scan('datalake/bronze/viajes/**/*.parquet', hive_partitioning=1)
""")

In [32]:
con.sql("""

    SELECT * FROM bronze.trips WHERE part_month = 1
    LIMIT 500
""").df()

,date,period,id_origin,id_destination,distance,activity_origin,activity_destination,study_origin_pos,study_destination_pos,residence,...,age,sex,n_trips,trips_total_length_km,source_url,ingestion_date,part_month,part_year,source,zone_type
0,2023-01-31,00,01001,01001,2-10,frecuente,casa,no,no,01,...,NA,NA,5.739,23.034,https://movilidad-opendata.mitma.es/estudios_b...,2025-11-25 17:29:18.482244,1,2023,MITMA,Distritos
1,2023-01-31,00,01001,01001,2-10,frecuente,trabajo_estudio,no,no,01,...,45-65,hombre,2.807,10.502,https://movilidad-opendata.mitma.es/estudios_b...,2025-11-25 17:29:18.482244,1,2023,MITMA,Distritos
2,2023-01-31,05,01001,01001,2-10,frecuente,casa,no,no,01,...,NA,NA,2.468,9.65,https://movilidad-opendata.mitma.es/estudios_b...,2025-11-25 17:29:18.482244,1,2023,MITMA,Distritos
3,2023-01-31,05,01001,01001,2-10,trabajo_estudio,frecuente,no,no,01,...,45-65,hombre,2.807,10.502,https://movilidad-opendata.mitma.es/estudios_b...,2025-11-25 17:29:18.482244,1,2023,MITMA,Distritos
4,2023-01-31,08,01001,01001,2-10,casa,no_frecuente,no,no,01,...,NA,NA,3.072,11.34,https://movilidad-opendata.mitma.es/estudios_b...,2025-11-25 17:29:18.482244,1,2023,MITMA,Distritos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2023-01-31,19,01001,0105903,10-50,no_frecuente,casa,no,no,01,...,45-65,hombre,2.807,36.136,https://movilidad-opendata.mitma.es/estudios_b...,2025-11-25 17:29:18.482244,1,2023,MITMA,Distritos
496,2023-01-31,19,01001,0105903,10-50,no_frecuente,casa,no,no,01,...,65-100,hombre,5.37,68.864,https://movilidad-opendata.mitma.es/estudios_b...,2025-11-25 17:29:18.482244,1,2023,MITMA,Distritos
497,2023-01-31,19,01001,0105903,10-50,trabajo_estudio,no_frecuente,no,no,01,...,25-45,hombre,4.093,51.697,https://movilidad-opendata.mitma.es/estudios_b...,2025-11-25 17:29:18.482244,1,2023,MITMA,Distritos
498,2023-01-31,20,01001,0105903,10-50,casa,frecuente,no,no,01,...,NA,NA,11.187,131.805,https://movilidad-opendata.mitma.es/estudios_b...,2025-11-25 17:29:18.482244,1,2023,MITMA,Distritos


In [30]:
def cargar_info_zona(con,zone_type):
    # zone_type = 'municipios','distritos','gaus'
    #Find in catalog zone type files
    file_info = con.sql(f"""
    SELECT source_url, filename 
    FROM bronze.catalog 
    WHERE (main_category = 'Zonificacion' OR main_category = 'Otros')
      AND (filename ILIKE '%{zone_type}%') 
      AND (filename ILIKE '%.shp' OR filename ILIKE '%.shx' OR filename ILIKE '%.dbf' OR filename ILIKE '%.prj' OR filename ILIKE '%.csv')
      AND filename NOT ILIKE '%centroides%'
        """).df()
    
    #temp dowload data
    urls = file_info["source_url"]
    os.makedirs(f"temp_downloads/{zone_type}", exist_ok=True)
    for key, url in urls.items():
        filename = url.split('/')[-1]
        path = f"temp_downloads/{zone_type}/{filename}"
        # Descargamos solo si no existe para ahorrar tiempo
        if not os.path.exists(path):
            print(f"Descargando {filename}...")
            r = requests.get(url)
            with open(path, 'wb') as f:
                f.write(r.content)

    zone_dic = {"municipios":"municiples","distritos":"districts","gaus":"GAUS"}


    shp_path = f"temp_downloads/{zone_type}/zonificacion_{zone_type}.shp"
    csv_path = f"temp_downloads/{zone_type}/nombres_{zone_type}.csv"
    output_parquet = f"datalake/bronze/zonificacion/{zone_dic[zone_type]}.parquet"
    os.makedirs("datalake/bronze/zonificacion", exist_ok=True)
    #Join tables into zonetype info table
    
    con.sql(f"""
            COPY (
                SELECT 
                    -- IDs y Nombres (Usamos el diccionario para acertar el nombre de columna)
                    CAST(t1.ID AS VARCHAR) as id_{zone_dic[zone_type]},
                    t2.name as name_{zone_dic[zone_type]},
                    
                    -- Geometría (Se guardará como binario WKB automáticamente)
                    t1.geom as geometry
                    
                FROM st_read('{shp_path}') t1
                -- Left Join con el CSV de nombres
                LEFT JOIN read_csv('{csv_path}', delim='|', header=True, auto_detect=True) t2 
                ON CAST(t1.ID AS VARCHAR) = CAST(t2.ID AS VARCHAR)
            ) 
            TO '{output_parquet}' 
            (FORMAT PARQUET, COMPRESSION 'ZSTD', OVERWRITE_OR_IGNORE)
        """)
    con.sql(f"""
            CREATE OR REPLACE VIEW bronze.{zone_dic[zone_type]}_info AS 
            SELECT 
                id_{zone_dic[zone_type]},
                name_{zone_dic[zone_type]},
                geometry
            FROM parquet_scan('{output_parquet}')
        """)
      

In [58]:
cargar_info_zona(con,"distritos")

In [57]:
con.sql("""

    SELECT * FROM bronze.municiples_info 

""").df()

,id_municiples,name_municiples,geometry
0,01001,Alegría-Dulantzi,"[5, 4, 0, 0, 0, 0, 0, 0, 109, 16, 3, 73, 121, ..."
1,01002,Amurrio,"[5, 4, 0, 0, 0, 0, 0, 0, 122, 90, 242, 72, 222..."
2,01004_AM,Artziniega agregacion de municipios,"[5, 4, 0, 0, 0, 0, 0, 0, 132, 235, 236, 72, 15..."
3,01009_AM,Asparrena agregacion de municipios,"[2, 4, 0, 0, 0, 0, 0, 0, 78, 113, 2, 73, 43, 1..."
4,01010,Ayala/Aiara,"[2, 4, 0, 0, 0, 0, 0, 0, 28, 17, 237, 72, 106,..."
...,...,...,...
2730,FRC21,None,"[2, 4, 0, 0, 0, 0, 0, 0, 156, 86, 141, 73, 137..."
2731,FRF11,None,"[2, 4, 0, 0, 0, 0, 0, 0, 123, 222, 149, 73, 70..."
2732,FRF33,None,"[2, 4, 0, 0, 0, 0, 0, 0, 60, 109, 139, 73, 37,..."
2733,FRI33,None,"[5, 4, 0, 0, 0, 0, 0, 0, 43, 7, 33, 73, 204, 1..."


In [68]:
file_info = con.sql(f"""
      SELECT source_url, filename 
      FROM bronze.catalog 
      WHERE (main_category = 'Zonificacion' OR main_category = 'Otros')
        AND (filename ILIKE '%relacion%') 
      
          
          """).df()
urls = file_info["source_url"][0]

con.sql(f"""
            COPY (
                SELECT *
                    
                FROM read_csv(
                    '{file_info['source_url'][0]}', 
                    delim='|', 
                    header=True, 
                    filename=True,
                    --union_by_name=True,
                    null_padding=True,
                    ignore_errors=True,
                    all_varchar=True -- Leemos todo como texto para evitar fallos de tipo antes de castear
                )
               
            ) 
            TO 'datalake/bronze/zonificacion/INE_MITMA_zone_relation.parquet' 
            (FORMAT PARQUET, COMPRESSION 'ZSTD', OVERWRITE_OR_IGNORE)
        """)

con.sql(f"""
            CREATE OR REPLACE VIEW bronze.INE_MITMA_zone_relation AS 
            SELECT *
              
            FROM parquet_scan('datalake/bronze/zonificacion/INE_MITMA_zone_relation.parquet')
        """)
      

In [69]:
con.sql("SELECT * FROM bronze.INE_MITMA_zone_relation")

┌─────────────┬──────────────┬───────────────┬────────────────┬─────────────────┬─────────────────┬─────────────────────────────────────────────────────────────────────────────────────┐
│ seccion_ine │ distrito_ine │ municipio_ine │ distrito_mitma │ municipio_mitma │    gau_mitma    │                                      filename                                       │
│   varchar   │   varchar    │    varchar    │    varchar     │     varchar     │     varchar     │                                       varchar                                       │
├─────────────┼──────────────┼───────────────┼────────────────┼─────────────────┼─────────────────┼─────────────────────────────────────────────────────────────────────────────────────┤
│ 0100101001  │ 0100101      │ 01001         │ 01001          │ 01001           │ 01001           │ https://movilidad-opendata.mitma.es/zonificacion/relacion_ine_zonificacionMitma.csv │
│ 0100101002  │ 0100101      │ 01001         │ 01001          │ 01001 

In [3]:
import duckdb
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import os
from pathlib import Path
import geopandas as gpd
import xml.etree.ElementTree as ET
import re
from pathlib import Path


In [31]:
con = duckdb.connect()
con.sql("INSTALL ducklake; LOAD ducklake;")
con.sql("INSTALL spatial; LOAD spatial;")

In [25]:
con.sql(f"""
USE memory;
DETACH my_ducklake;
    """)

In [32]:
con.sql(f"""
ATTACH 'ducklake:my_ducklake.ducklake' AS my_ducklake;

USE my_ducklake;
    """)

In [5]:
con.sql("CREATE SCHEMA IF NOT EXISTS bronze")

In [12]:
con.sql("CREATE OR REPLACE TABLE bronze.catalog AS SELECT * FROM df_catalog")

In [20]:
files_df = con.sql("""
    SELECT *
    FROM bronze.catalog 
   
""").df()
files_df

,main_category,study_type,zone_type,year,month,day,publication_date,filename,source_url
0,Estudios Completos,Viajes,N/A,2023.0,12.0,17.0,2025-11-18 12:15:51,20231217_viajes.csv.gz,https://movilidad-opendata.mitma.es/estudios_c...
1,Estudios Completos,Viajes,N/A,2023.0,12.0,16.0,2025-11-18 12:15:50,20231216_viajes.csv.gz,https://movilidad-opendata.mitma.es/estudios_c...
2,Estudios Completos,Viajes,N/A,2023.0,12.0,13.0,2025-11-18 12:15:46,20231213_viajes.csv.gz,https://movilidad-opendata.mitma.es/estudios_c...
3,Estudios Completos,Viajes,N/A,2023.0,12.0,14.0,2025-11-18 12:15:46,20231214_viajes.csv.gz,https://movilidad-opendata.mitma.es/estudios_c...
4,Estudios Completos,Viajes,N/A,2023.0,12.0,15.0,2025-11-18 12:15:46,20231215_viajes.csv.gz,https://movilidad-opendata.mitma.es/estudios_c...
...,...,...,...,...,...,...,...,...,...
13378,Estudios Basicos,Desconocido,Distritos,NaN,NaN,NaN,2022-12-20 15:07:40,,https://movilidad-opendata.mitma.es/estudios_b...
13379,Estudios Basicos,Calidad,N/A,NaN,NaN,NaN,2022-12-20 15:00:05,,https://movilidad-opendata.mitma.es/estudios_b...
13380,Estudios Basicos,Calidad,N/A,NaN,NaN,NaN,2022-12-20 14:59:42,,https://movilidad-opendata.mitma.es/estudios_b...
13381,Estudios Basicos,Desconocido,N/A,NaN,NaN,NaN,2022-12-20 14:59:32,,https://movilidad-opendata.mitma.es/estudios_b...


In [33]:
con.sql("""
    SELECT * FROM bronze.trips WHERE part_month = '2'
""")

┌─────────┬─────────┬───────────┬────────────────┬──────────┬─────────────────┬──────────────────────┬──────────────────┬───────────────────────┬───────────┬─────────┬─────────┬─────────┬─────────┬───────────────────────┬───────────┬────────────┬───────────┬─────────┬────────────┬────────────────┐
│  date   │ period  │ id_origin │ id_destination │ distance │ activity_origin │ activity_destination │ study_origin_pos │ study_destination_pos │ residence │  rent   │   age   │   sex   │ n_trips │ trips_total_length_km │ part_year │ part_month │ zone_type │ source  │ source_url │ ingestion_date │
│ varchar │ varchar │  varchar  │    varchar     │ varchar  │     varchar     │       varchar        │     varchar      │        varchar        │  varchar  │ varchar │ varchar │ varchar │ varchar │        varchar        │  varchar  │  varchar   │  varchar  │ varchar │  varchar   │   timestamp    │
├─────────┴─────────┴───────────┴────────────────┴──────────┴─────────────────┴──────────────────────┴─

In [27]:
duckdb.sql(f"""
   FROM glob('my_ducklake.ducklake.files/**/*');
    FROM 'my_ducklake.ducklake.files/**/*.parquet' LIMIT 10;    
    """)

┌──────────┬──────────────────────┬───────────┬─────────┬──────────────────────┬──────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [46]:
con.sql("""
    SELECT * FROM bronze.catalog
""")

┌────────────────────┬────────────┬───────────┬────────┬────────┬────────┬─────────────────────┬────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│   main_category    │ study_type │ zone_type │  year  │ month  │  day   │  publication_date   │                filename                │                                                                 source_url                                                                  │
│      varchar       │  varchar   │  varchar  │ double │ double │ double │    timestamp_ns     │                varchar                 │                                                                   varchar                                                                   │
├────────────────────┼────────────┼───────────┼────────┼────────┼────────┼─────────────────────┼────────────────────────────────────────┼───────────────────────

In [37]:
con.sql("DROP TABLE bronze.trips")

In [38]:
con.sql("""
    CREATE OR REPLACE TABLE bronze.trips (
        date VARCHAR,
        period VARCHAR,
        id_origin VARCHAR,
        id_destination VARCHAR,
        distance VARCHAR,
        activity_origin VARCHAR,
        activity_destination VARCHAR,
        study_origin_pos VARCHAR,
        study_destination_pos VARCHAR,
        residence VARCHAR,
        rent VARCHAR,
        age VARCHAR,
        sex VARCHAR,
        n_trips VARCHAR,
        trips_total_length_km VARCHAR,
        
        part_year VARCHAR,
        part_month VARCHAR,
        zone_type VARCHAR,
        source VARCHAR,
        source_url VARCHAR,
        ingestion_date TIMESTAMP
    );
""")
# con.sql("ALTER TABLE bronze.trips SET PARTITIONED BY (zone_type, source, part_year, part_month)")


In [ ]:


def cargar_viajes_por_criterio(con, year, month, zone_type):
    # zone_type format : 'Municipio' 'Distritos' 'GAUS'
    files_df = con.sql(f"""
        SELECT source_url 
        FROM bronze.catalog 
        WHERE year = {year}
        AND month = {month}
        AND zone_type = '{zone_type}'
        AND main_category = 'Estudios Basicos'
        AND study_type = 'Viajes'
        AND filename LIKE '%.csv.gz'
    """).df()

    urls = files_df['source_url'].tolist()
    print(urls)
    con.sql(f"""INSERT INTO bronze.trips
            SELECT 
                        -- 1. Transformación de Fecha (Corrección del VARCHAR '20230131')
                        fecha as date,
                        periodo as period,
                        origen as id_origin,
                        destino as id_destination ,
                        distancia as distance,
                        actividad_origen as activity_origin,
                        actividad_destino as activity_destination,
                        estudio_origen_posible as study_origin_pos,
                        estudio_destino_posible as study_destination_pos,
                        residencia as residence,
                        renta as rent,
                        edad as age,
                        sexo as sex,
                        viajes as n_trips,
                        viajes_km as trips_total_length_km,
                        -- 2. Metadatos (Tus columnas personalizadas)
                        {year} as part_year,
                        {month} as part_month,
                        '{zone_type}' as zone_type,  -- Valor fijo pasado por parámetro
                        'MITMA' as source,
                        filename as source_url,      -- Columna mágica generada por read_csv(..., filename=True)
                        current_timestamp as ingestion_date
                        
                    FROM read_csv(
                        {urls}, 
                        delim='|', 
                        header=True, 
                        filename=True,
                        union_by_name=True,
                        null_padding=True,
                        ignore_errors=True,
                        all_varchar=True -- Leemos todo como texto para evitar fallos de tipo antes de castear
                    )""")

In [49]:
cargar_viajes_por_criterio(con, 2023, 2, "Distritos")

['https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/viajes/ficheros-diarios/2023-02/20230228_Viajes_distritos.csv.gz', 'https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/viajes/ficheros-diarios/2023-02/20230227_Viajes_distritos.csv.gz', 'https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/viajes/ficheros-diarios/2023-02/20230226_Viajes_distritos.csv.gz', 'https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/viajes/ficheros-diarios/2023-02/20230225_Viajes_distritos.csv.gz', 'https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/viajes/ficheros-diarios/2023-02/20230224_Viajes_distritos.csv.gz', 'https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/viajes/ficheros-diarios/2023-02/20230223_Viajes_distritos.csv.gz', 'https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/viajes/ficheros-diarios/2023-02/20230222_Viajes_distritos.csv.gz', 'https://movilidad-opendata.mitma.es/estudios_basicos/

In [61]:
con.sql("""
    SELECT * FROM bronze.trips WHERE zone_type = 'Distritos' AND date = '20230101'  and period = '00' and id_origin = '38039' -- and activity_origin = 'casa' and activity_destination = 'frecuente' and id_destination = '24122_AM'
    ORDER BY n_trips DESC

""")

┌──────────┬─────────┬───────────┬────────────────┬──────────┬─────────────────┬──────────────────────┬──────────────────┬───────────────────────┬───────────┬─────────┬─────────┬─────────┬─────────┬───────────────────────┬───────────┬────────────┬───────────┬─────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────┐
│   date   │ period  │ id_origin │ id_destination │ distance │ activity_origin │ activity_destination │ study_origin_pos │ study_destination_pos │ residence │  rent   │   age   │   sex   │ n_trips │ trips_total_length_km │ part_year │ part_month │ zone_type │ source  │                                                             source_url                                                              │       ingestion_date       │
│ varchar  │ varchar │  varchar  │    varchar     │ varchar  │     varchar     │       varchar        │     varchar      │        varc

In [6]:
con.sql("""
    SELECT * FROM bronze.trips WHERE part_month = '2'

""")

CatalogException: Catalog Error: Table with name trips does not exist!
Did you mean "information_schema.views"?

In [27]:
con.sql(f"""
    CREATE OR REPLACE TABLE bronze.{zone_dic[zone_type]}_info (
        id_{zone_dic[zone_type]} VARCHAR,
        name_{zone_dic[zone_type]} VARCHAR,
        geometry VARCHAR,
        zone_type VARCHAR,
        source VARCHAR,
        source_url VARCHAR,
        ingestion_date TIMESTAMP
    );
""")
con.sql(f"ALTER TABLE bronze.{zone_dic[zone_type]}_info SET PARTITIONED BY (zone_type)")

NameError: name 'zone_dic' is not defined

In [16]:
def cargar_info_zona(con,zone_type):
    # zone_type = 'municipios','distritos','gaus'
    #Find in catalog zone type files
    zone_dic = {"municipios":"municiples","distritos":"districts","gaus":"GAUS"}

    con.sql(f"""
    CREATE OR REPLACE TABLE bronze.{zone_dic[zone_type]}_info (
        id_{zone_dic[zone_type]} VARCHAR,
        name_{zone_dic[zone_type]} VARCHAR,
        zone_type VARCHAR,
        source VARCHAR,
        source_url VARCHAR,
        ingestion_date TIMESTAMP,
        geometry GEOMETRY ,
        centroid GEOMETRY ,
            );
        """)
    con.sql(f"ALTER TABLE bronze.{zone_dic[zone_type]}_info SET PARTITIONED BY (zone_type)")
    
    file_info = con.sql(f"""
    SELECT source_url, filename 
    FROM bronze.catalog 
    WHERE (main_category = 'Zonificacion' OR main_category = 'Otros')
      AND (filename ILIKE '%{zone_type}%') 
      AND (filename ILIKE '%.shp' OR filename ILIKE '%.shx' OR filename ILIKE '%.dbf' OR filename ILIKE '%.prj' OR filename ILIKE '%.csv')
      --AND filename NOT ILIKE '%centroides%'
        """).df()
    print(len(file_info))
    #temp dowload data
    urls = file_info["source_url"]
    os.makedirs(f"temp_downloads/{zone_type}", exist_ok=True)
    for key, url in urls.items():
        filename = url.split('/')[-1]
        path = f"temp_downloads/{zone_type}/{filename}"
        # Descargamos solo si no existe para ahorrar tiempo
        if not os.path.exists(path):
            print(f"Descargando {filename}...")
            r = requests.get(url)
            with open(path, 'wb') as f:
                f.write(r.content)




    shp_path = f"temp_downloads/{zone_type}/zonificacion_{zone_type}.shp"
    shp_centroid_path = f"temp_downloads/{zone_type}/zonificacion_{zone_type}_centroides.shp"
    csv_path = f"temp_downloads/{zone_type}/nombres_{zone_type}.csv"
    output_parquet = f"datalake/bronze/zonificacion/{zone_dic[zone_type]}.parquet"
    os.makedirs("datalake/bronze/zonificacion", exist_ok=True)
    #Join tables into zonetype info table
    
    con.sql(f"""
            INSERT INTO bronze.{zone_dic[zone_type]}_info
                SELECT 
                    -- IDs y Nombres (Usamos el diccionario para acertar el nombre de columna)
                    CAST(t1.ID AS VARCHAR) as id_{zone_dic[zone_type]},
                    t2.name as name_{zone_dic[zone_type]},
                    
                    -- Geometría (Se guardará como binario WKB automáticamente)

                    '{zone_dic[zone_type]}' as zone_type,
                    'MITMA' as source,
                    '{urls}' as source_url,      -- Columna mágica generada por read_csv(..., filename=True)
                    current_timestamp as ingestion_date,
                    t1.geom as geometry,
                    t3.geom as centroid
                FROM st_read('{shp_path}') t1
                -- Left Join con el CSV de nombres
                LEFT JOIN read_csv('{csv_path}', delim='|', header=True, auto_detect=True,filename=True) t2 
                ON CAST(t1.ID AS VARCHAR) = CAST(t2.ID AS VARCHAR)
                LEFT JOIN st_read('{shp_centroid_path}') t3 ON CAST(t1.ID AS VARCHAR) = CAST(t3.ID AS VARCHAR)
            
           """)
      

In [19]:
cargar_info_zona(con,"gaus")
cargar_info_zona(con,"municipios")
cargar_info_zona(con,"distritos")

10
10
10


In [18]:
con.sql("DROP TABLE bronze.GAUS_info ")
con.sql("DROP TABLE bronze.municiples_info ")
con.sql("DROP TABLE bronze.districts_info ")

In [20]:
con.sql("SELECT * FROM bronze.districts_info WHERE name_districts LIKE '%Ali%'").df()

,id_districts,name_districts,zone_type,source,source_url,ingestion_date,geometry,centroid
0,0301401,Alicante/Alacant distrito 01,districts,MITMA,0 https://movilidad-opendata.mitma.es/zonif...,2025-11-26 14:24:53.243352,"[2, 4, 0, 0, 0, 0, 0, 0, 157, 99, 47, 73, 135,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,0301402,Alicante/Alacant distrito 02,districts,MITMA,0 https://movilidad-opendata.mitma.es/zonif...,2025-11-26 14:24:53.243352,"[2, 4, 0, 0, 0, 0, 0, 0, 233, 133, 47, 73, 215...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,0301403,Alicante/Alacant distrito 03,districts,MITMA,0 https://movilidad-opendata.mitma.es/zonif...,2025-11-26 14:24:53.243352,"[2, 4, 0, 0, 0, 0, 0, 0, 97, 178, 47, 73, 172,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,0301404,Alicante/Alacant distrito 04,districts,MITMA,0 https://movilidad-opendata.mitma.es/zonif...,2025-11-26 14:24:53.243352,"[5, 4, 0, 0, 0, 0, 0, 0, 125, 161, 46, 73, 108...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,0301405,Alicante/Alacant distrito 05,districts,MITMA,0 https://movilidad-opendata.mitma.es/zonif...,2025-11-26 14:24:53.243352,"[2, 4, 0, 0, 0, 0, 0, 0, 243, 87, 47, 73, 182,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,0301406,Alicante/Alacant distrito 06,districts,MITMA,0 https://movilidad-opendata.mitma.es/zonif...,2025-11-26 14:24:53.243352,"[2, 4, 0, 0, 0, 0, 0, 0, 216, 180, 46, 73, 183...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6,0301407,Alicante/Alacant distrito 07,districts,MITMA,0 https://movilidad-opendata.mitma.es/zonif...,2025-11-26 14:24:53.243352,"[2, 4, 0, 0, 0, 0, 0, 0, 217, 146, 46, 73, 205...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
7,0301408,Alicante/Alacant distrito 08,districts,MITMA,0 https://movilidad-opendata.mitma.es/zonif...,2025-11-26 14:24:53.243352,"[5, 4, 0, 0, 0, 0, 0, 0, 73, 10, 44, 73, 252, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [22]:
import duckdb
import geopandas as gpd
import pandas as pd
from shapely import wkb


con.sql("INSTALL spatial; LOAD spatial;")
# 1. Extraemos los datos convirtiendo la geometría a WKB
# Nota: Filtramos por una provincia (ej. Madrid '28%') para no saturar el mapa al principio
query = """
    SELECT 
        id_districts,
        name_districts,

        geometry as geom_wkb
    FROM bronze.districts_info
    WHERE name_districts LIKE '%Ali%'
"""

df = con.sql(query).df()


# 2. Convertimos el DataFrame normal a GeoDataFrame
# Decodificamos el binario WKB a objetos geométricos reales
df['geometry'] = df['geom_wkb']
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# 3. Definir el Sistema de Referencia de Coordenadas (CRS)
# Los datos del MITMA suelen venir en ETRS89 (EPSG:4258)
gdf.set_crs(epsg=4258, inplace=True)

print(f"Cargados {len(gdf)} municipios.")
import matplotlib.pyplot as plt

# Visualización simple
fig, ax = plt.subplots(figsize=(10, 15))
gdf.plot(ax=ax, legend=True, cmap='OrRd', edgecolor='black', linewidth=0.1)
ax.set_title("Municipios (Mapa Estático)")
plt.show()

TypeError: Input must be valid geometry objects: bytearray(b'\x02\x04\x00\x00\x00\x00\x00\x00\x9dc/I\x87\x8d\x81J\x96\xfb/I+\xa3\x81J\x02\x00\x00\x00\x01\x00\x00\x00i\x08\x00\x00\x00\x00\x00\x00\xb8b\x7fy\xf0\xfb%A\xf2A\xcf2y3PA \xf4l6\xf0\xfb%A\xe4\x14\x1d\x19y3PA\x90\x87\x85\xda\xed\xfb%A\xf6\x97\xdd[x3PA\x88\xfde\x17\xed\xfb%A\x0cq\xac\xdbw3PAPt$\xb7\xea\xfb%A\x02\x9a\x08/w3PA\x90uq\xdb\xe6\xfb%A\x02\x9a\x08_v3PA\x084\x116\xe4\xfb%A\xbaI\x0c\x82u3PA(\x06\x81U\xe4\xfb%A\xcc\xcc\xcc\xb4t3PA Zd\xdb\xe5\xfb%A6\xcd;2t3PA\xc0t\x93\xf8\xe6\xfb%A\xbe\x0e\x9c\xdbs3PA\x18\xd9\xce\xb7\xe6\xfb%A\xea\x95\xb2\x94s3PA \xfd\xf65\xe4\xfb%A\x8e\x06\xf0^s3PA \xfd\xf6\x95\xdf\xfb%A>\n\xd7+s3PA\x00"\xfd\xb6\xdc\xfb%A\xee|?\ts3PA\x10\xd0Dx\xda\xfb%A\xcc\xcc\xcc\xd4r3PAHY\x86\xf8\xd8\xfb%A\xe4\xa5\x9btr3PA\xb8b\x7f\xd9\xd5\xfb%AP\x1e\x16\xc2q3PA \xf4l\xb6\xd2\xfb%A\xac\xad\xd8[q3PA lx\xda\xcf\xfb%A\xf6\xb9\xda\x8ep3PA\xf0\x85\xc9T\xcb\xfb%AvO\x1e\xf2m3PA\x10X9T\xce\xfb%ARI\x9d\x04m3PA\xf0\xfd\xd4x\xd6\xfb

In [125]:
con.sql("SELECT * FROM __ducklake_metadata_my_ducklake.ducklake_data_file")

┌──────────────┬──────────┬────────────────┬──────────────┬────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────┬─────────────┬──────────────┬─────────────────┬─────────────┬──────────────┬──────────────┬────────────────┬───────────────────┬────────────┐
│ data_file_id │ table_id │ begin_snapshot │ end_snapshot │ file_order │                                                        path                                                         │ path_is_relative │ file_format │ record_count │ file_size_bytes │ footer_size │ row_id_start │ partition_id │ encryption_key │ partial_file_info │ mapping_id │
│    int64     │  int64   │     int64      │    int64     │   int64    │                                                       varchar                                                       │     boolean      │   varchar   │    int64     │      int64      │    int64    │    int64     │    int64    

In [91]:
con.sql("FROM my_ducklake.snapshots();")

┌─────────────┬───────────────────────────────┬────────────────┬─────────────────────────────────────────────────────────────────┬─────────┬────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                             changes                             │ author  │ commit_message │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                     map(varchar, varchar[])                     │ varchar │    varchar     │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼─────────────────────────────────────────────────────────────────┼─────────┼────────────────┼───────────────────┤
│           0 │ 2025-11-26 00:22:11.288769+01 │              0 │ {schemas_created=[main]}                                        │ NULL    │ NULL           │ NULL              │
│           1 │ 2025-11-26 00:22:14.81608+01  │              1 │ {schemas_created=[bronze]}                   